

**Jupyter notebook quick start:**<br>
- \<Shift>+\<Return> --> runs a cell
- ! --> shell escape [! Linux command line]
- use a (above) and b (below) left of the [ ] to open new cells
---

# Discrete Fourier Transform

The discrete Fourier transform (DFT) is a fundamental transform in digital signal processing.

It is used to analyze the frequency content of a signal. The DFT is defined as:
$$
X(k) = Σ x(n) * exp(-j2πnk/N)
$$ 

where $X(k)$ is the $k$th frequency component of the signal,
$x(n)$ is the nth sample of the signal, 
$j$ is the imaginary unit, and
$N$ is the number of samples in the signal.

The DFT is a complex-valued function, and it returns the amplitude and phase of each frequency component in the signal.

# Serial vs Parallel DFT

The Python program below finds the maximum frequency in the given signal and also shows the computation time required for the calculation.

In [4]:
%%writefile exercises/DFT_serial.py

import numpy as np
import time
import matplotlib.pyplot as plt


def dft(x): # Discrete Fourier Transform
    N = len(x)
    X = []
    for k in range(N//2+1):
        X.append(sum(x[n]*np.exp(-2j*np.pi*k*n/N) for n in range(N)))
    return X

# Start time
start_time = time.time()

# Data
samp = 1000
dt = 1/samp
t = np.arange(0, 5, dt)
data = np.sin(150*2*np.pi*t)+2*np.sin(250*2*np.pi*t)+3*np.sin(350*2*np.pi*t) #signal

n = len(data)
N2 = n // 2
amp = dft(data) # Discrete Fourier Transform
frq = np.fft.fftfreq(n, dt) # Calculate the frequency

# End time
end_time = time.time()

# Calculation time
calc_time = end_time - start_time
print(f'Calculation time: {calc_time:.2f} seconds')

amp = np.abs(amp) # Calculate absolute value of the amplitude

# Find the maximum amplitude and its corresponding frequency
maxA = np.max(amp[:N2])
maxF = frq[np.argmax(amp[:N2])]
print(f'Max frq = {maxF:.2f} \n')

plt.plot(frq[:N2], amp[:N2]) # Plot the amplitude spectrum
plt.savefig('amplitude.png')

Writing exercises/DFT_serial.py


In [6]:
! python exercises/DFT_serial.py

Calculation time: 26.59 seconds
Max frq = 350.00 



It can be seen that finding the maximum frequency takes quite a long time.

Let's apply the learned MPI standard to make the calculation faster!

In [9]:
%%writefile exercises/DFT_parallel.py

import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI

def dft(x): # Discrete Fourier Transform
    N = len(x)
    X = []
    for k in range(N//2+1):
        X.append(sum(x[n]*np.exp(-2j*np.pi*k*n/N) for n in range(N)))
    return X

#for calculation time
tinit = MPI.Wtime()

#MPI for DFT
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#Data
samp = 1000
dt = 1/samp
t = np.arange(0, 5, dt)
if rank == 0:
    data = np.sin(150*2*np.pi*t)+2*np.sin(250*2*np.pi*t)+3*np.sin(350*2*np.pi*t) #signal
    data = np.array_split(data, size) #split the data into the number of processes
else:
    data = None

#Scatter data to all processes and calculate DFT
data = comm.scatter(data, root=0)
amp = dft(data)

#Calculate the frequency
f = np.fft.fftfreq(len(data), dt)

# End time
tend = MPI.Wtime()

#Calculate absolute value of the amplitude
amp = np.abs(amp)

#Show the results
if rank == 0:
    print(f'Parallel Time: {tend-tinit:.2f} s')
    N2 = len(data)//2 #take only positive frequencies
    maxA = np.max(amp[:N2]) #maximum amplitude
    maxF = f[np.argmax(amp[:N2])] #corresponding frequency
    print(f'Max frq = {maxF:.2f} \n')

    plt.plot(f[1:N2],amp[1:N2]) #plot the amplitude spectrum
    plt.xlabel('Hz')
    plt.savefig('amplitude2.png')
    

Writing exercises/DFT_parallel.py


In [11]:
! mpiexec -n 2 python exercises/DFT_parallel.py

Parallel Time: 7.21 s
Max frq = 350.00 



---
<img src="img/exercise.png" width="45"> **Exercise**

Find the optimal number of parallel processes!

---